In [317]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import os
import time

In [318]:
base_link = "https://www.themoviedb.org/movie"

In [319]:
needed_headers = {'User-Agent': "Mozilla/5.0 (Windows NT 6.3; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/92.0.4515.131 Safari/537.36"}
response = requests.get("https://www.themoviedb.org/movie", headers = needed_headers )

In [320]:
response.status_code

200

In [321]:
dwn_content = response.text
len(dwn_content)

195373

In [322]:
test_doc = BeautifulSoup(response.text, 'html.parser')

In [323]:
test_doc.find('title')

<title>Popular Movies — The Movie Database (TMDB)</title>

In [324]:
test_doc.find('img')

<img alt="The Movie Database (TMDB)" height="20" src="/assets/2/v4/logos/v2/blue_short-8e7b30f73a4020692ccca9c88bafe5dcb6f8a62a4c6bc55cd9ba82bb2cd95f6c.svg" width="154"/>

In [325]:
def cols_dict():
    dic = {  
            'genres': [],
            'language': [], 
            'title': [], 
            'rating': [],
            'runtime': [],
            'cast': [],
            'release_date': [],
            'director': [],
            'Revenue' : [],
            'Budget': [],
            'Keywords': [],
            'type': []
          }
    return dic

In [326]:
def get_page_content(url):
    # In this case , we are going to give request.get function headers to avoid the Status code Error 403

    get_headers = {'User-Agent': "Mozilla/5.0 (Windows NT 6.3; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/92.0.4515.131 Safari/537.36"}
    response_page = requests.get(url, headers = get_headers )
    # we are going to raise exception here if status code gives any value other than 200.
    if not response_page.ok:
        raise Exception ("Failed to request the data. Status Code:- {}".format(response_page.status_code))
    else:
        page_content = response_page.text
        doc_page = BeautifulSoup(page_content, "html.parser")
        return doc_page

In [327]:
def get_genres(doc2):
    tag_genre = doc2.find('span', {"class": "genres"})
    tag_genre_list = tag_genre.find_all('a')

    check_genre =[]
    for tag in tag_genre_list:
        check_genre.append(tag.text)

    return check_genre

In [328]:
def get_cast(doc2_page):
    cast_tags = doc2_page.find_all('li', {'class': 'card'})
    cast_lis = []
    
    for t in cast_tags:
         cast_lis.append(t.p.text)
    
    return cast_lis

In [329]:
def get_Keywords(doc3_page):
    Keywords_tags = doc3_page.find('section', {'class': 'keywords right_column'})
    Keywords_list = Keywords_tags.find_all('a')
    
    check_Keywords = []
    for t in Keywords_list:
         check_Keywords.append(t.text)
    
    return Keywords_list

In [330]:
def get_Keywords(doc3_page):
    Keywords_tags = doc3_page.find('section', {'class': 'keywords right_column'})
    Keywords_list = Keywords_tags.find_all('a')
    
    check_Keywords = []
    for t in Keywords_list:
         check_Keywords.append(t.text)
    
    return Keywords_list

In [331]:
%%time
url_address = base_link

dic =  cols_dict()
for j in range(1,501):
    next_url = url_address + '?page={}'.format(j)
    get_headers = {'User-Agent': "Mozilla/5.0 (Windows NT 6.3; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/92.0.4515.131 Safari/537.36"}
    response_page = requests.get(next_url, headers = get_headers )
    # we are going to raise exception here if status code gives any value other than 200.
    if not response_page.ok:
        print('page: ',j, "Failed to request the data. Status Code:- {}".format(response_page.status_code))
    else:
        page_content = response_page.text
        doc = BeautifulSoup(page_content, "html.parser")
    
        number_of_movies = len(doc.find_all('div', {'class': 'card style_1'}))
        for i in range(number_of_movies): 

            try:
                dic['title'].append(doc.find_all('div', {'class': 'card style_1'})[i].h2.text)
            except AttributeError:
                dic['title'].append('nan')

            try:
                dic['rating'].append(doc.find_all('div', {"user_score_chart"})[i]['data-percent'])
            except AttributeError:
                dic['rating'].append('nan')

            try:
                dic['release_date'].append(doc.find_all('div', {'class': 'card style_1'})[i].p.text)
            except AttributeError:
                dic['release_date'].append('nan')

            movie_url = base_link[:-6] + doc.find_all('div', {'class': 'card style_1'})[i].h2.a['href']
            movie_page = get_page_content(movie_url)

            dic['genres'].append(get_genres(movie_page))
            dic['cast'].append(get_cast(movie_page))
            dic['Keywords'].append(get_Keywords(movie_page))
            dic['type'].append('movie')
                
            try:
                dic['runtime'].append(movie_page.find('span', {"class": "runtime"}).text[16:-12])
            except AttributeError:
                dic['runtime'].append('nan')

            try:
                dic['Revenue'].append(movie_page.find('section', {"class": "facts left_column"}).select("p:nth-of-type(4)")[0].text)
            except AttributeError:
                dic['Revenue'].append('nan')
                
            try:
                dic['language'].append(movie_page.find('section', {"class": "facts left_column"}).select("p:nth-of-type(2)")[0].text)
            except AttributeError:
                dic['language'].append('nan')
                
            try:
                dic['director'].append(movie_page.find('li', {'class':'profile'}).a.get_text())
            except AttributeError:
                dic['director'].append('nan')
                
            try:
                dic['Budget'].append(movie_page.find('section', {"class": "facts left_column"}).select("p:nth-of-type(3)")[0].text)
            except AttributeError:
                dic['Budget'].append('nan')
    

Wall time: 2h 21min 25s


In [332]:
j,i

(500, 19)

In [333]:
df_00 = pd.DataFrame({k:pd.Series(v) for k,v in dic.items()})

In [334]:
df_00.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 12 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   genres        10000 non-null  object
 1   language      10000 non-null  object
 2   title         10000 non-null  object
 3   rating        10000 non-null  object
 4   runtime       10000 non-null  object
 5   cast          10000 non-null  object
 6   release_date  10000 non-null  object
 7   director      10000 non-null  object
 8   Revenue       10000 non-null  object
 9   Budget        10000 non-null  object
 10  Keywords      10000 non-null  object
 11  type          10000 non-null  object
dtypes: object(12)
memory usage: 937.6+ KB


In [335]:
df_00

,genres,language,title,rating,runtime,cast,release_date,director,Revenue,Budget,Keywords,type
0,"[Action, Adventure, Science Fiction]",Original Language English,Spider-Man: No Way Home,85.0,2h 28m,"[Tom Holland, Zendaya, Benedict Cumberbatch, J...","Dec 15, 2021",Stan Lee,"Revenue $1,694,026,010.00","Budget $200,000,000.00","[[hero], [comic book], [superhero], [based on ...",movie
1,"[Action, Adventure, Fantasy, Science Fiction]",Original Language English,Eternals,73.0,2h 36m,"[Gemma Chan, Richard Madden, Angelina Jolie, K...","Nov 03, 2021",Chloé Zhao,"Revenue $401,972,153.00","Budget $200,000,000.00","[[boredom], [supernatural], [superhero], [base...",movie
2,"[Animation, Family, Fantasy, Comedy, Adventure]",Original Language English,Hotel Transylvania: Transformania,77.0,1h 27m,"[Selena Gomez, Andy Samberg, Kathryn Hahn, Jim...","Jan 13, 2022",Genndy Tartakovsky,Revenue -,Budget -,[[vampire]],movie
3,"[Animation, Comedy, Family, Fantasy]",Original Language English,Encanto,78.0,1h 42m,"[Stephanie Beatriz, María Cecilia Botero, John...","Nov 24, 2021",Jared Bush,"Revenue $228,000,000.00","Budget $50,000,000.00","[[magic], [musical], [forest], [family relatio...",movie
4,"[Horror, Action, Science Fiction]",Original Language English,Resident Evil: Welcome to Raccoon City,61.0,1h 47m,"[Kaya Scodelario, Robbie Amell, Hannah John-Ka...","Nov 24, 2021",Johannes Roberts,"Revenue $31,000,000.00","Budget $40,000,000.00","[[mutant], [biological weapon], [quarantine], ...",movie
...,...,...,...,...,...,...,...,...,...,...,...,...
9995,"[Horror, Mystery, Thriller]",Original Language English,Cherry Falls,51.0,1h 32m,"[Brittany Murphy, Jay Mohr, Michael Biehn, Jes...","Aug 25, 2000",Geoffrey Wright,Revenue -,Budget -,"[[rape], [small town], [virgin], [psychopath],...",movie
9996,"[Horror, Thriller]",Original Language English,Inside,52.0,1h 25m,"[Rachel Nichols, Laura Harring, Ben Temple, Gi...","Feb 14, 2016",Miguel Ángel Vivas,Revenue -,Budget -,[[remake]],movie
9997,[Action],\nStatus Released\n,Cutie Honey: Tears,56.0,1h 32m,"[Mariya Nishiuchi, Takahiro Miura, Nicole Ishi...","Oct 01, 2016",Takeshi Asai,Budget -,Original Language Japanese,"[[fight], [based on manga], [battle], [cutie h...",movie
9998,"[Adventure, Fantasy, Comedy]",Original Language English,Erik the Viking,59.0,1h 47m,"[Tim Robbins, Mickey Rooney, Eartha Kitt, Terr...","Sep 01, 1989",Terry Jones,Revenue -,Budget -,"[[norway], [viking], [based on children's book...",movie


In [336]:
df_00.to_csv('movies_all.csv', index =False)

In [337]:
df_01 = pd.read_csv('movies_all.csv')
df_01.shape

(10000, 12)

In [338]:
df_01.head()

,genres,language,title,rating,runtime,cast,release_date,director,Revenue,Budget,Keywords,type
0,"['Action', 'Adventure', 'Science Fiction']",Original Language English,Spider-Man: No Way Home,85.0,2h 28m,"['Tom Holland', 'Zendaya', 'Benedict Cumberbat...","Dec 15, 2021",Stan Lee,"Revenue $1,694,026,010.00","Budget $200,000,000.00","[<a href=""/keyword/1701-hero/movie"">hero</a>, ...",movie
1,"['Action', 'Adventure', 'Fantasy', 'Science Fi...",Original Language English,Eternals,73.0,2h 36m,"['Gemma Chan', 'Richard Madden', 'Angelina Jol...","Nov 03, 2021",Chloé Zhao,"Revenue $401,972,153.00","Budget $200,000,000.00","[<a href=""/keyword/3925-boredom/movie"">boredom...",movie
2,"['Animation', 'Family', 'Fantasy', 'Comedy', '...",Original Language English,Hotel Transylvania: Transformania,77.0,1h 27m,"['Selena Gomez', 'Andy Samberg', 'Kathryn Hahn...","Jan 13, 2022",Genndy Tartakovsky,Revenue -,Budget -,"[<a href=""/keyword/3133-vampire/movie"">vampire...",movie
3,"['Animation', 'Comedy', 'Family', 'Fantasy']",Original Language English,Encanto,78.0,1h 42m,"['Stephanie Beatriz', 'María Cecilia Botero', ...","Nov 24, 2021",Jared Bush,"Revenue $228,000,000.00","Budget $50,000,000.00","[<a href=""/keyword/2343-magic/movie"">magic</a>...",movie
4,"['Horror', 'Action', 'Science Fiction']",Original Language English,Resident Evil: Welcome to Raccoon City,61.0,1h 47m,"['Kaya Scodelario', 'Robbie Amell', 'Hannah Jo...","Nov 24, 2021",Johannes Roberts,"Revenue $31,000,000.00","Budget $40,000,000.00","[<a href=""/keyword/1852-mutant/movie"">mutant</...",movie
